In [552]:
# !pip install altair
# !pip install altair_saver --upgrade
# !npm install -g vega-lite vega-cli canvas
# !pip install vl-convert-python --upgrade

In [553]:
# !pip install selenium --upgrade
# !apt-get install chromium-chromedriver -y

In [554]:
# !apt update
# !apt install ttf-mscorefonts-installer -y
# !apt reinstall fontconfig fontconfig-config libfontconfig1 -y

In [555]:
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Black.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Bold.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Book.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Medium.otf -P /usr/local/share/fonts

In [556]:
# !fc-cache -f

In [557]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import urllib.parse

In [558]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [659]:
SAVE = True
LOCAL = False
DARK = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [660]:
%%capture pwd
!pwd

In [661]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
uid=urllib.parse.quote(uid)
if not LOCAL:
    eco_git_home = (
        "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
    )
    vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
    colors = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
    )
    category_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
    )
    hue_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
    )
    mhue_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
    )
    div_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
    )
    config = json.loads(
        requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
    )
else:
    eco_git_home = '/'.join(pwd.stdout.split("/")[:-2])+'/'
    vega_embed = open(eco_git_home + "guidelines/html/vega-embed.html",'r').read()
    colors = json.load(
        open(eco_git_home + "guidelines/colors/eco-colors.json",'r')
    )
    category_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-category-color.json",'r')
    )
    hue_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-single-hue-color.json",'r')
    )
    mhue_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-multi-hue-color.json",'r')
    )
    div_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-diverging-color.json",'r')
    )
    config = json.load(
        open(eco_git_home + "guidelines/charts/eco-global-config.json",'r')
    )
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
mo=0.5
height = config["height"]
width = config["width"]
uid, height, width

('how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero',
 300,
 500)

In [662]:
def save(df, f, LOCAL):
    fc = eco_git_path + f + ".csv"
    df.to_csv("data/" + f + ".csv")
    f += local_suffix
    open("visualisation/" + f + ".html", "w").write(
        vega_embed.replace(
            "JSON_PATH", fc.replace("/data/", "/visualisation/").replace(".csv", ".json")
        )
    )
    if LOCAL:
        fc = df
    
    from IPython.display import display, HTML
    display(HTML(df.head().to_html()))
    
    readme = "## Figure " + f.replace('fig','').split('_')[0] + \
        '  \n\nData: [`csv`](data/' + f + '.csv)' +\
        '  \nGitHub: [' + f + '](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/'+uid +')'+\
        ''+\
        '  \n\n### Light theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local.png) [`svg`](visualisation/' + f + '_local.svg) [`json`](visualisation/' + f + '_local.json) '+ \
        '  \n (**Default**) Versions with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '.png) [`svg`](visualisation/' + f + '.svg) [`json`](visualisation/' + f + '.json)'+ \
        '  \nVersions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_no_branding.png) [`svg`](visualisation/' + f + '_local_no_branding.svg) [`json`](visualisation/' + f + '_local_no_branding.json) '+ \
        '  \nVersions (no ECO branding) with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_no_branding.png) [`svg`](visualisation/' + f + '_no_branding.svg) [`json`](visualisation/' + f + '_no_branding.json) '+ \
        ''+\
        '  \n\n### Dark theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_dark.png) [`svg`](visualisation/' + f + '_local_dark.svg) [`json`](visualisation/' + f + '_local_dark.json) '+ \
        '  \n Versions with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_dark.png) [`svg`](visualisation/' + f + '_dark.svg) [`json`](visualisation/' + f + '_dark.json)'+ \
        '  \nVersions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_no_branding_dark.png) [`svg`](visualisation/' + f + '_local_no_branding_dark.svg) [`json`](visualisation/' + f + '_local_no_branding_dark.json) '+ \
        '  \nVersions (no ECO branding) with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_no_branding_dark.png) [`svg`](visualisation/' + f + '_no_branding_dark.svg) [`json`](visualisation/' + f + '_no_branding_dark.json) '+ \
        ''+\
        '  \n\n!["' + f + '"](visualisation/' + f + '.svg "' + f + '")\n\n' +\
        '  \n\n!["' + f + '_dark"](visualisation/' + f + '_dark.svg "' + f + '")\n\n' 
    return readme, f, fc

In [663]:
def area(base,color,opacity=1):
    return base.mark_area(opacity=opacity,
    interpolate="monotone",
    line={'color':color},
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='#ffffff00', offset=0.2),
               alt.GradientStop(color=color, offset=0.8)],
        x1=1, #0.8
        y1=1,
        x2=1,
        y2=0
        )
    )

In [664]:
# service_color='#d6c8da' '#e4bfe2' '#ce4b96' colors['eco-turquiose']
service_color='#b4c8d8'
def dark(f):
    configSource = "visualisation/" + f + ".json"
    config = json.loads(open(configSource, "r").read())
    config['background']=colors['eco-background']
    service_color='#b4c8d8'
    for i in config['layer']:
        if 'encoding' in i:
            for x in ['x','y']:
                if x in i['encoding']:
                    if 'axis' in i['encoding'][x]:
                        if i['encoding'][x]['axis']:
                            for c in ['domainColor','labelColor','tickColor','titleColor','gridColor']:
                                if c in i['encoding'][x]['axis']:
                                    i['encoding'][x]['axis'][c]=service_color
        if 'mark' in i:
            if 'color' in i['mark']:
                if i['mark']['color']==colors['eco-gray']:
                    i['mark']['color']=service_color
                elif i['mark']['color']==colors['eco-blue']:
                    i['mark']['color']=colors['eco-yellow']
                elif i['mark']['color']==service_color:
                    i['mark']['color']=colors['eco-green']
                elif 'stops' in i['mark']['color']:
                    for s in i['mark']['color']['stops']:
                        if 'color' in s:
                            if s['color']==colors['eco-gray']:
                                s['color']=service_color
                            elif s['color']==colors['eco-blue']:
                                s['color']=colors['eco-yellow']
                            elif s['color']==service_color:
                                s['color']=colors['eco-green']
            if 'line' in i['mark']:
                if 'color' in i['mark']['line']:
                    if i['mark']['line']['color']==colors['eco-gray']:
                        i['mark']['line']['color']=service_color
                    elif i['mark']['line']['color']==colors['eco-blue']:
                        i['mark']['line']['color']=colors['eco-yellow']
                    elif i['mark']['line']['color']==service_color:
                        i['mark']['line']['color']=colors['eco-green']
    if 'datasets' in config:
        for i in config['datasets']:
            if 'img' in config['datasets'][i][0]:
                if 'eco-icon-dark' in config['datasets'][i][0]['img']:
                    config['datasets'][i][0]['img']=config['datasets'][i][0]['img'].replace('eco-icon-dark','eco-icon-light')
    return alt.Chart.from_dict(config) 

# Fig 1

In [665]:
df = pd.read_csv("raw/gmt_HadCRUT5.csv")
df=(df.set_index('Year')[['HadCRUT5 (degC)']]-0.35).rename(columns={'HadCRUT5 (degC)':'t'}).reset_index()

In [666]:
df

,Year,t
0,1850,-0.43
1,1851,-0.24
2,1852,-0.24
3,1853,-0.28
4,1854,-0.30
...,...,...
170,2020,0.91
171,2021,0.75
172,2022,0.79
173,2023,1.09


In [667]:
readme, f, fc = save(df,"fig1_temp",LOCAL)

,Year,t
0,1850,-0.43
1,1851,-0.24
2,1852,-0.24
3,1853,-0.28
4,1854,-0.30


In [668]:
xmin=1850
xmax=2024
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            orient="bottom",
            labelAngle=0,
        ),
        scale=alt.Scale(domain=[1849, 2025],nice=False)
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Temperature anomaly in  ° C",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            # format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[1850, 2025]),
    )
)

base = alt.Chart(fc).encode(x='Year:Q')
line1=base.mark_line(color=colors['eco-light-blue'],interpolate='basis').encode(y='t:Q')
label1=line1.mark_text(text='Reference: mean for 1960-90',fontSize=12,color=colors['eco-gray'],align='left',opacity=mo+0.2,dy=-75,dx=5).transform_filter("datum.Year==1850")

layer1 = (
    (xaxis+line1+label1).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:Q',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "w").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig1_temp



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 2

In [678]:
readme, f, fc = save(df,"fig2_stripes",LOCAL)

,Year,t
0,1990,8715.849181
1,1991,8193.454903
2,1992,8183.456325
3,1993,8596.475342
4,1994,8881.201695


In [680]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=list(range(1860, 2021, 20)),
            titleAlign="left",
            titleAnchor="start",
            title="Deviation from 1960-1990 average world temperature",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=3).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[-0.6,0.9],
        scheme='redblue',
        reverse=True,
        bins=[-0.6,-0.3,0,0.3,0.6,0.9,1]
    )).legend(title="",labelColor=colors["eco-gray"]),
        tooltip=['Year:N','Anomaly:N']).transform_calculate(Anomaly='round(datum.t*100)/100')

layer1 = (
    (bars1+bars1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig2_stripes



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 3

In [681]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df=df[df['Country Name']=='World'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [682]:
readme, f, fc = save(df,"fig3_gdp",LOCAL)

,Year,t
0,1990,9705.981267
1,1991,9669.67706
2,1992,9665.89026
3,1993,9675.23226
4,1994,9799.764965


In [683]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=list(range(1990, 2025, 5)),
            titleAlign="left",
            titleAnchor="start",
            title="GDP per capita at PPP in 2017 international constant US dollars",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=12).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[10000,17000],
        scheme='redblue',
        reverse=True,
        bins=[10000,12000,13500,15000,17000,19000]
    )).legend(title="",labelColor=colors["eco-gray"]),
        tooltip=['Year:N','GDP:N']).transform_calculate(GDP='round(datum.t*100)/100')

layer1 = (
    (bars1+bars1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig3_gdp



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 4

In [684]:
df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df=df[df['Country Name']=='World'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [685]:
readme, f, fc = save(df,"fig4_co2",LOCAL)

,Year,t
0,1990,4.020868
1,1991,3.983343
2,1992,3.91031
3,1993,3.874985
4,1994,3.842027


In [686]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=list(range(1990, 2025, 5)),
            titleAlign="left",
            titleAnchor="start",
            title="Carbon dioxide emissions per capita in metric tons",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=13).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[3.8,4.6],
        scheme='brownbluegreen',
        reverse=True,
        bins=[3.8,4.0,4.2,4.4,4.6,4.7]
    )).legend(title="",labelColor=colors["eco-gray"]),
        tooltip=['Year:N','Emissions:N']).transform_calculate(Emissions='round(datum.t*100)/100')

layer1 = (
    (bars1+bars1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig4_co2



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 5

In [687]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df1=df[df['Country Name']=='World'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna()
df1.columns=['t']

df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df2=df[df['Country Name']=='World'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna()
df2.columns=['t']

df=(df1/df2).dropna().reset_index()
df.columns=['Year','t']

In [688]:
readme, f, fc = save(df,"fig5_gpr",LOCAL)

,Year,t
0,1990,2413.902178
1,1991,2427.528167
2,1992,2471.89853
3,1993,2496.843635
4,1994,2550.67551


In [689]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=list(range(1990, 2025, 5)),
            titleAlign="left",
            titleAnchor="start",
            title="Greening prosperity per capita in discounted 2017 US dollars",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=13).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[2600,3800],
        scheme='brownbluegreen',
        # reverse=True,
        bins=[2600,3000,3400,3800,3900]
    )).legend(title="",labelColor=colors["eco-gray"]),
        tooltip=['Year:N','GPR:N']).transform_calculate(GPR='round(datum.t*100)/100')

layer1 = (
    (bars1+bars1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig5_gpr



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 6

## a

In [690]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df=df[df['Country Name']=='World'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [691]:
readme, f, fc = save(df,"fig6a_gdp_world",LOCAL)

,Year,t
0,1990,9705.981267
1,1991,9669.67706
2,1992,9665.89026
3,1993,9675.23226
4,1994,9799.764965


In [692]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=list(range(1990, 2025, 5)),
            titleAlign="left",
            titleAnchor="start",
            title="GDP per capita at PPP in 2017 international constant US dollars",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=12).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[1000,100000],
        scheme='redblue',
        reverse=True,
        bins=[1000,50000,100000,190000]
    )).legend(title="",labelColor=colors["eco-gray"]),
        tooltip=['Year:N','GDP:N']).transform_calculate(GDP='round(datum.t*100)/100')
label1=bars1.mark_text(text='World',fill='white',fontSize=12,align='left',dy=-60).transform_filter('datum.Year==1990')

layer1 = (
    (label1+bars1+bars1+label1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig6a_gdp_world



alt.LayerChart(...)

alt.LayerChart(...)


## b

In [693]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df=df[df['Country Name']=='Switzerland'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [694]:
readme, f, fc = save(df,"fig6b_gdp_ch",LOCAL)

,Year,t
0,1990,56232.02608
1,1991,55025.011308
2,1992,54397.880761
3,1993,53836.808853
4,1994,54087.519552


In [695]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=list(range(1990, 2025, 5)),
            titleAlign="left",
            titleAnchor="start",
            title="GDP per capita at PPP in 2017 international constant US dollars",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=12).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[1000,100000],
        scheme='redblue',
        reverse=True,
        bins=[1000,50000,100000,190000]
    )).legend(title="",labelColor=colors["eco-gray"]),
        tooltip=['Year:N','GDP:N']).transform_calculate(GDP='round(datum.t*100)/100')
label1=bars1.mark_text(text='Switzerland',fill=colors["eco-gray"],fontSize=12,align='left',dy=-60).transform_filter('datum.Year==1990')

layer1 = (
    (label1+bars1+bars1+label1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig6b_gdp_ch



alt.LayerChart(...)

alt.LayerChart(...)

## c

In [696]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df=df[df['Country Name']=='Mozambique'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [697]:
readme, f, fc = save(df,"fig6b_gdp_mz",LOCAL)

,Year,t
0,1990,460.123697
1,1991,473.612004
2,1992,436.376416
3,1993,469.397817
4,1994,476.370759


In [698]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=list(range(1990, 2025, 5)),
            titleAlign="left",
            titleAnchor="start",
            title="GDP per capita at PPP in 2017 international constant US dollars",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=12).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[1000,100000],
        scheme='redblue',
        reverse=True,
        bins=[1000,50000,100000,190000]
    )).legend(title="",labelColor=colors["eco-gray"]),
        tooltip=['Year:N','GDP:N']).transform_calculate(GDP='round(datum.t*100)/100')
label1=bars1.mark_text(text='Mozambique',fill='white',fontSize=12,align='left',dy=-60).transform_filter('datum.Year==1990')

layer1 = (
    (label1+bars1+bars1+label1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig6b_gdp_mz



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 7

## a

In [699]:
df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df=df[df['Country Name']=='World'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [700]:
readme, f, fc = save(df,"fig7_co2_world",LOCAL)

,Year,t
0,1990,4.020868
1,1991,3.983343
2,1992,3.91031
3,1993,3.874985
4,1994,3.842027


In [701]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=list(range(1990, 2025, 5)),
            titleAlign="left",
            titleAnchor="start",
            title="Carbon dioxide emissions per capita in metric tons",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=13).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[1,20],
        scheme='brownbluegreen',
        reverse=True,
        bins=[1, 10, 20, 21]
    )).legend(title="",labelColor=colors["eco-gray"]),
        tooltip=['Year:N','Emissions:N']).transform_calculate(Emissions='round(datum.t*100)/100')
label1=bars1.mark_text(text='World',fill='white',fontSize=12,align='left',dy=-60).transform_filter('datum.Year==1990')

layer1 = (
    (bars1+bars1+label1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig7_co2_world



alt.LayerChart(...)

alt.LayerChart(...)

## b

In [702]:
df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df=df[df['Country Name']=='Switzerland'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [703]:
readme, f, fc = save(df,"fig7b_co2_ch",LOCAL)

,Year,t
0,1990,6.451698
1,1991,6.715728
2,1992,6.647299
3,1993,6.262661
4,1994,6.090113


In [704]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=list(range(1990, 2025, 5)),
            titleAlign="left",
            titleAnchor="start",
            title="Carbon dioxide emissions per capita in metric tons",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=13).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[1,20],
        scheme='brownbluegreen',
        reverse=True,
        bins=[1, 10, 20, 21]
    )).legend(title="",labelColor=colors["eco-gray"]),
        tooltip=['Year:N','Emissions:N']).transform_calculate(Emissions='round(datum.t*100)/100')
label1=bars1.mark_text(text='Switzerland',fill='white',fontSize=12,align='left',dy=-60).transform_filter('datum.Year==1990')

layer1 = (
    (bars1+bars1+label1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig7b_co2_ch



alt.LayerChart(...)

alt.LayerChart(...)

## c

In [705]:
df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df=df[df['Country Name']=='Mozambique'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [706]:
readme, f, fc = save(df,"fig7c_co2_mz",LOCAL)

,Year,t
0,1990,0.08398
1,1991,0.069027
2,1992,0.079423
3,1993,0.090705
4,1994,0.073553


In [707]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=list(range(1990, 2025, 5)),
            titleAlign="left",
            titleAnchor="start",
            title="Carbon dioxide emissions per capita in metric tons",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=13).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[1,20],
        scheme='brownbluegreen',
        reverse=True,
        bins=[1, 10, 20, 21]
    )).legend(title="",labelColor=colors["eco-gray"]),
        tooltip=['Year:N','Emissions:N']).transform_calculate(Emissions='round(datum.t*100)/100')
label1=bars1.mark_text(text='Mozambique',fill='white',fontSize=12,align='left',dy=-60).transform_filter('datum.Year==1990')

layer1 = (
    (bars1+bars1+label1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig7c_co2_mz



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 8

## a

In [708]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df1=df[df['Country Name']=='World'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna()
df1.columns=['t']

df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df2=df[df['Country Name']=='World'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna()
df2.columns=['t']

df=(df1/df2).dropna().reset_index()
df.columns=['Year','t']

In [709]:
readme, f, fc = save(df,"fig8a_gpr_world",LOCAL)

,Year,t
0,1990,2413.902178
1,1991,2427.528167
2,1992,2471.89853
3,1993,2496.843635
4,1994,2550.67551


In [710]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=list(range(1990, 2025, 5)),
            titleAlign="left",
            titleAnchor="start",
            title="Greening prosperity per capita in discounted 2017 US dollars",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=13).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[1000,20000],
        scheme='brownbluegreen',
        # reverse=True,
        bins=[1000,8000,14000,20000,23900]
    )).legend(title="",labelColor=colors["eco-gray"]),
        tooltip=['Year:N','GPR:N']).transform_calculate(GPR='round(datum.t*100)/100')
label1=bars1.mark_text(text='World',fill='white',fontSize=12,align='left',dy=-60).transform_filter('datum.Year==1990')

layer1 = (
    (bars1+bars1+label1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig8a_gpr_world



alt.LayerChart(...)

alt.LayerChart(...)

## b

In [711]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df1=df[df['Country Name']=='Switzerland'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna()
df1.columns=['t']

df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df2=df[df['Country Name']=='Switzerland'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna()
df2.columns=['t']

df=(df1/df2).dropna().reset_index()
df.columns=['Year','t']

In [712]:
readme, f, fc = save(df,"fig8b_gpr_ch",LOCAL)

,Year,t
0,1990,8715.849181
1,1991,8193.454903
2,1992,8183.456325
3,1993,8596.475342
4,1994,8881.201695


In [713]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=list(range(1990, 2025, 5)),
            titleAlign="left",
            titleAnchor="start",
            title="Greening prosperity per capita in discounted 2017 US dollars",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=13).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[1000,20000],
        scheme='brownbluegreen',
        # reverse=True,
        bins=[1000,8000,14000,20000,23900]
    )).legend(title="",labelColor=colors["eco-gray"]),
        tooltip=['Year:N','GPR:N']).transform_calculate(GPR='round(datum.t*100)/100')
label1=bars1.mark_text(text='Switzerland',fill=colors["eco-gray"],fontSize=12,align='left',dy=-60).transform_filter('datum.Year==1990')

layer1 = (
    (bars1+bars1+label1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig8b_gpr_ch



alt.LayerChart(...)

alt.LayerChart(...)

## c

In [714]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df1=df[df['Country Name']=='Mozambique'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna()
df1.columns=['t']

df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df2=df[df['Country Name']=='Mozambique'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna()
df2.columns=['t']

df=(df1/df2).dropna().reset_index()
df.columns=['Year','t']

In [715]:
readme, f, fc = save(df,"fig8c_gpr_mz",LOCAL)

,Year,t
0,1990,5478.940535
1,1991,6861.236916
2,1992,5494.323758
3,1993,5175.015014
4,1994,6476.603298


In [716]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=list(range(1990, 2025, 5)),
            titleAlign="left",
            titleAnchor="start",
            title="Greening prosperity per capita in discounted 2017 US dollars",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=13).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[1000,20000],
        scheme='brownbluegreen',
        # reverse=True,
        bins=[1000,8000,14000,20000,23900]
    )).legend(title="",labelColor=colors["eco-gray"]),
        tooltip=['Year:N','GPR:N']).transform_calculate(GPR='round(datum.t*100)/100')
label1=bars1.mark_text(text='Mozambique',fill='white',fontSize=12,align='left',dy=-60).transform_filter('datum.Year==1990')

layer1 = (
    (bars1+bars1+label1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig8c_gpr_mz



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 9

## a

In [717]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df=df.set_index('Country Code').drop(['Country Name',
                                     'Indicator Name','Indicator Code'],axis=1)['2020'].dropna()
df=df.loc[df.index.dropna()].reset_index()
df.columns=['Country','t']

In [718]:
readme, f, fc = save(df,"fig9a_gdp_map",LOCAL)

,Country,t
0,ABW,33155.243239
1,AFE,3455.023119
2,AFG,1968.341002
3,AFW,3957.933804
4,AGO,6029.691895


In [719]:
# countries = alt.topo_feature('https://raw.githubusercontent.com/jhellingsdata/RADataHub/main/e4e/ch1_GDP/data/chart1_4_world_map_gdp_world_countries.geojson', 'features')
countries = alt.Data(url='https://raw.githubusercontent.com/jhellingsdata/RADataHub/main/e4e/ch1_GDP/data/chart1_4_world_map_gdp_world_countries.geojson', 
                     format=alt.DataFormat(property='features',type='json'))

layer1=alt.Chart(countries).mark_geoshape().encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[1000,100000],
        scheme='redblue',
        reverse=True,
        bins=[1000,50000,100000,190000]
    )).legend(title="",labelColor=colors["eco-gray"],legendY=120),
        tooltip=['Country:N','GDP:N']).transform_lookup(
    lookup='properties.ISO3',
    from_=alt.LookupData(df, 'Country', ['Country','t'])
).project(
    type='naturalEarth1'
).transform_calculate(GDP='round(datum.t*100)/100')

logo=alt.Chart(pd.DataFrame([{"x": 0, "y": 0, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-160,opacity=mo,xOffset=500).encode(x=alt.X('x:O',axis=None),y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)

layer1+=(logo)

layer1=layer1.properties(height=300, width=500, title={'text':"GDP per capita at PPP in 2020, in 2017 international constant US dollars",
                                                'color':'#9597AA'
                                                })\
    .configure(font='Circular Std Book').configure_view(stroke=None)
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
        
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()

fig9a_gdp_map



alt.LayerChart(...)

## b

In [720]:
df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df=df.set_index('Country Code').drop(['Country Name',
                                     'Indicator Name','Indicator Code'],axis=1)['2020'].dropna()
df=df.loc[df.index.dropna()].reset_index()
df.columns=['Country','t']

In [721]:
readme, f, fc = save(df,"fig9b_co2_map",LOCAL)

,Country,t
0,ABW,1.000000
1,AFE,0.795420
2,AFG,0.223479
3,AFW,0.463150
4,AGO,0.592743


In [722]:
# countries = alt.topo_feature('https://raw.githubusercontent.com/jhellingsdata/RADataHub/main/e4e/ch1_GDP/data/chart1_4_world_map_gdp_world_countries.geojson', 'features')
countries = alt.Data(url='https://raw.githubusercontent.com/jhellingsdata/RADataHub/main/e4e/ch1_GDP/data/chart1_4_world_map_gdp_world_countries.geojson', 
                     format=alt.DataFormat(property='features',type='json'))

layer1=alt.Chart(countries).mark_geoshape().encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[1,20],
        scheme='brownbluegreen',
        reverse=True,
        bins=[1, 10, 20, 21]
    )).legend(title="",labelColor=colors["eco-gray"],legendY=120),
        tooltip=['Country:N','Emissions:N']).transform_lookup(
    lookup='properties.ISO3',
    from_=alt.LookupData(df, 'Country', ['Country','t'])
).project(
    type='naturalEarth1'
).transform_calculate(Emissions='round(datum.t*100)/100')

logo=alt.Chart(pd.DataFrame([{"x": 0, "y": 0, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-160,opacity=mo,xOffset=500).encode(x=alt.X('x:O',axis=None),y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

layer1=layer1.properties(height=300, width=500, title={'text':"Carbon dioxide emissions per capita in 2020 in metric tons",
                                                'color':'#9597AA'
                                                })\
    .configure(font='Circular Std Book').configure_view(stroke=None)

if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
        
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()

fig9b_co2_map



alt.LayerChart(...)

## c

In [723]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df=df.set_index('Country Code').drop(['Country Name',
                                     'Indicator Name','Indicator Code'],axis=1)['2020'].dropna()
df1=df.loc[df.index.dropna()].reset_index()
df1.columns=['Country','t']

df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df=df.set_index('Country Code').drop(['Country Name',
                                     'Indicator Name','Indicator Code'],axis=1)['2020'].dropna()
df2=df.loc[df.index.dropna()].reset_index()
df2.columns=['Country','t']

df=(df1.set_index('Country')/df2.set_index('Country')).reset_index()

In [724]:
readme, f, fc = save(df,"fig9c_gpr_map",LOCAL)

,Country,t
0,ABW,33155.243239
1,AFE,4343.646774
2,AFG,8807.727477
3,AFW,8545.679518
4,AGO,10172.523712


In [725]:
# countries = alt.topo_feature('https://raw.githubusercontent.com/jhellingsdata/RADataHub/main/e4e/ch1_GDP/data/chart1_4_world_map_gdp_world_countries.geojson', 'features')
countries = alt.Data(url='https://raw.githubusercontent.com/jhellingsdata/RADataHub/main/e4e/ch1_GDP/data/chart1_4_world_map_gdp_world_countries.geojson', 
                     format=alt.DataFormat(property='features',type='json'))

layer1=alt.Chart(countries).mark_geoshape().encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[1000,20000],
        scheme='brownbluegreen',
        # reverse=True,
        bins=[1000,8000,14000,20000,23900]
    )).legend(title="",labelColor=colors["eco-gray"],legendY=120),
        tooltip=['Country:N','Emissions:N']).transform_lookup(
    lookup='properties.ISO3',
    from_=alt.LookupData(df, 'Country', ['Country','t'])
).project(
    type='naturalEarth1'
).transform_calculate(Emissions='round(datum.t*100)/100')

logo=alt.Chart(pd.DataFrame([{"x": 0, "y": 0, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-160,opacity=mo,xOffset=500).encode(x=alt.X('x:O',axis=None),y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)

layer1+=(logo)

layer1=layer1.properties(height=300, width=500, title={'text':"Greening prosperity per capita in 2020 in discounted 2017 US dollars",
                                                'color':'#9597AA'
                                                })\
    .configure(font='Circular Std Book').configure_view(stroke=None)

if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
        
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()

fig9c_gpr_map



alt.LayerChart(...)

# Post-process

## Clean up `SVG`s
`base64 URI` encode images

In [726]:
from base64 import b64encode

In [727]:
from os import listdir
from os.path import isfile, join
mypath='./visualisation/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
svgs=[i for i in onlyfiles if i[-4:]=='.svg']

In [728]:
for svg in svgs:
    s=open(mypath+svg,'r').read()
    si=s.find('"image mark" xlink:href=')
    if si>-1:
        s2=s[si+25:]
        imgurl=s2[:s2.find('"')]
        if 'data:image/png;' not in imgurl:
            s1=s[:si+25]+s2.replace(imgurl,'data:image/png;base64,'+b64encode(requests.get(imgurl).content).decode('utf-8'))
            open(mypath+svg,'w').write(s1)
            print(svg)

fig1_temp.svg
fig1_temp_dark.svg
fig1_temp_local.svg
fig1_temp_local_dark.svg
fig2_stripes.svg
fig2_stripes_dark.svg
fig2_stripes_local.svg
fig2_stripes_local_dark.svg
fig3_gdp.svg
fig3_gdp_dark.svg
fig3_gdp_local.svg
fig3_gdp_local_dark.svg
fig4_co2.svg
fig4_co2_dark.svg
fig4_co2_local.svg
fig4_co2_local_dark.svg
fig5_gpr.svg
fig5_gpr_dark.svg
fig5_gpr_local.svg
fig5_gpr_local_dark.svg
fig6a_gdp_world.svg
fig6a_gdp_world_dark.svg
fig6a_gdp_world_local.svg
fig6a_gdp_world_local_dark.svg
fig6b_gdp_ch.svg
fig6b_gdp_ch_dark.svg
fig6b_gdp_ch_local.svg
fig6b_gdp_ch_local_dark.svg
fig6b_gdp_mz.svg
fig6b_gdp_mz_dark.svg
fig6b_gdp_mz_local.svg
fig6b_gdp_mz_local_dark.svg
fig7b_co2_ch.svg
fig7b_co2_ch_dark.svg
fig7b_co2_ch_local.svg
fig7b_co2_ch_local_dark.svg
fig7c_co2_mz.svg
fig7c_co2_mz_dark.svg
fig7c_co2_mz_local.svg
fig7c_co2_mz_local_dark.svg
fig7_co2_world.svg
fig7_co2_world_dark.svg
fig7_co2_world_local.svg
fig7_co2_world_local_dark.svg
fig8a_gpr_world.svg
fig8a_gpr_world_dark.svg
fig8a

## README

In [729]:
from IPython.display import display, Markdown

with open('README.md', 'r') as fh:
    content = fh.read()

display(Markdown(content))

## Figure 1  

Data: [`csv`](data/fig1_temp.csv)  
GitHub: [fig1_temp](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_temp_local.png) [`svg`](visualisation/fig1_temp_local.svg) [`json`](visualisation/fig1_temp_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig1_temp.png) [`svg`](visualisation/fig1_temp.svg) [`json`](visualisation/fig1_temp.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_temp_local_no_branding.png) [`svg`](visualisation/fig1_temp_local_no_branding.svg) [`json`](visualisation/fig1_temp_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig1_temp_no_branding.png) [`svg`](visualisation/fig1_temp_no_branding.svg) [`json`](visualisation/fig1_temp_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_temp_local_dark.png) [`svg`](visualisation/fig1_temp_local_dark.svg) [`json`](visualisation/fig1_temp_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig1_temp_dark.png) [`svg`](visualisation/fig1_temp_dark.svg) [`json`](visualisation/fig1_temp_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_temp_local_no_branding_dark.png) [`svg`](visualisation/fig1_temp_local_no_branding_dark.svg) [`json`](visualisation/fig1_temp_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig1_temp_no_branding_dark.png) [`svg`](visualisation/fig1_temp_no_branding_dark.svg) [`json`](visualisation/fig1_temp_no_branding_dark.json)   

!["fig1_temp"](visualisation/fig1_temp.svg "fig1_temp")

  

!["fig1_temp_dark"](visualisation/fig1_temp_dark.svg "fig1_temp")

## Figure 2  

Data: [`csv`](data/fig2_stripes.csv)  
GitHub: [fig2_stripes](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_stripes_local.png) [`svg`](visualisation/fig2_stripes_local.svg) [`json`](visualisation/fig2_stripes_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig2_stripes.png) [`svg`](visualisation/fig2_stripes.svg) [`json`](visualisation/fig2_stripes.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_stripes_local_no_branding.png) [`svg`](visualisation/fig2_stripes_local_no_branding.svg) [`json`](visualisation/fig2_stripes_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig2_stripes_no_branding.png) [`svg`](visualisation/fig2_stripes_no_branding.svg) [`json`](visualisation/fig2_stripes_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_stripes_local_dark.png) [`svg`](visualisation/fig2_stripes_local_dark.svg) [`json`](visualisation/fig2_stripes_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig2_stripes_dark.png) [`svg`](visualisation/fig2_stripes_dark.svg) [`json`](visualisation/fig2_stripes_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_stripes_local_no_branding_dark.png) [`svg`](visualisation/fig2_stripes_local_no_branding_dark.svg) [`json`](visualisation/fig2_stripes_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig2_stripes_no_branding_dark.png) [`svg`](visualisation/fig2_stripes_no_branding_dark.svg) [`json`](visualisation/fig2_stripes_no_branding_dark.json)   

!["fig2_stripes"](visualisation/fig2_stripes.svg "fig2_stripes")

  

!["fig2_stripes_dark"](visualisation/fig2_stripes_dark.svg "fig2_stripes")

## Figure 3  

Data: [`csv`](data/fig3_gdp.csv)  
GitHub: [fig3_gdp](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_gdp_local.png) [`svg`](visualisation/fig3_gdp_local.svg) [`json`](visualisation/fig3_gdp_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig3_gdp.png) [`svg`](visualisation/fig3_gdp.svg) [`json`](visualisation/fig3_gdp.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_gdp_local_no_branding.png) [`svg`](visualisation/fig3_gdp_local_no_branding.svg) [`json`](visualisation/fig3_gdp_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig3_gdp_no_branding.png) [`svg`](visualisation/fig3_gdp_no_branding.svg) [`json`](visualisation/fig3_gdp_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_gdp_local_dark.png) [`svg`](visualisation/fig3_gdp_local_dark.svg) [`json`](visualisation/fig3_gdp_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig3_gdp_dark.png) [`svg`](visualisation/fig3_gdp_dark.svg) [`json`](visualisation/fig3_gdp_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_gdp_local_no_branding_dark.png) [`svg`](visualisation/fig3_gdp_local_no_branding_dark.svg) [`json`](visualisation/fig3_gdp_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig3_gdp_no_branding_dark.png) [`svg`](visualisation/fig3_gdp_no_branding_dark.svg) [`json`](visualisation/fig3_gdp_no_branding_dark.json)   

!["fig3_gdp"](visualisation/fig3_gdp.svg "fig3_gdp")

  

!["fig3_gdp_dark"](visualisation/fig3_gdp_dark.svg "fig3_gdp")

## Figure 4  

Data: [`csv`](data/fig4_co2.csv)  
GitHub: [fig4_co2](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_co2_local.png) [`svg`](visualisation/fig4_co2_local.svg) [`json`](visualisation/fig4_co2_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig4_co2.png) [`svg`](visualisation/fig4_co2.svg) [`json`](visualisation/fig4_co2.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_co2_local_no_branding.png) [`svg`](visualisation/fig4_co2_local_no_branding.svg) [`json`](visualisation/fig4_co2_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig4_co2_no_branding.png) [`svg`](visualisation/fig4_co2_no_branding.svg) [`json`](visualisation/fig4_co2_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_co2_local_dark.png) [`svg`](visualisation/fig4_co2_local_dark.svg) [`json`](visualisation/fig4_co2_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig4_co2_dark.png) [`svg`](visualisation/fig4_co2_dark.svg) [`json`](visualisation/fig4_co2_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_co2_local_no_branding_dark.png) [`svg`](visualisation/fig4_co2_local_no_branding_dark.svg) [`json`](visualisation/fig4_co2_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig4_co2_no_branding_dark.png) [`svg`](visualisation/fig4_co2_no_branding_dark.svg) [`json`](visualisation/fig4_co2_no_branding_dark.json)   

!["fig4_co2"](visualisation/fig4_co2.svg "fig4_co2")

  

!["fig4_co2_dark"](visualisation/fig4_co2_dark.svg "fig4_co2")

## Figure 5  

Data: [`csv`](data/fig5_gpr.csv)  
GitHub: [fig5_gpr](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_gpr_local.png) [`svg`](visualisation/fig5_gpr_local.svg) [`json`](visualisation/fig5_gpr_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig5_gpr.png) [`svg`](visualisation/fig5_gpr.svg) [`json`](visualisation/fig5_gpr.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_gpr_local_no_branding.png) [`svg`](visualisation/fig5_gpr_local_no_branding.svg) [`json`](visualisation/fig5_gpr_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig5_gpr_no_branding.png) [`svg`](visualisation/fig5_gpr_no_branding.svg) [`json`](visualisation/fig5_gpr_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_gpr_local_dark.png) [`svg`](visualisation/fig5_gpr_local_dark.svg) [`json`](visualisation/fig5_gpr_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig5_gpr_dark.png) [`svg`](visualisation/fig5_gpr_dark.svg) [`json`](visualisation/fig5_gpr_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_gpr_local_no_branding_dark.png) [`svg`](visualisation/fig5_gpr_local_no_branding_dark.svg) [`json`](visualisation/fig5_gpr_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig5_gpr_no_branding_dark.png) [`svg`](visualisation/fig5_gpr_no_branding_dark.svg) [`json`](visualisation/fig5_gpr_no_branding_dark.json)   

!["fig5_gpr"](visualisation/fig5_gpr.svg "fig5_gpr")

  

!["fig5_gpr_dark"](visualisation/fig5_gpr_dark.svg "fig5_gpr")

## Figure 6a  

Data: [`csv`](data/fig6a_gdp_world.csv)  
GitHub: [fig6a_gdp_world](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6a_gdp_world_local.png) [`svg`](visualisation/fig6a_gdp_world_local.svg) [`json`](visualisation/fig6a_gdp_world_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig6a_gdp_world.png) [`svg`](visualisation/fig6a_gdp_world.svg) [`json`](visualisation/fig6a_gdp_world.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6a_gdp_world_local_no_branding.png) [`svg`](visualisation/fig6a_gdp_world_local_no_branding.svg) [`json`](visualisation/fig6a_gdp_world_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6a_gdp_world_no_branding.png) [`svg`](visualisation/fig6a_gdp_world_no_branding.svg) [`json`](visualisation/fig6a_gdp_world_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6a_gdp_world_local_dark.png) [`svg`](visualisation/fig6a_gdp_world_local_dark.svg) [`json`](visualisation/fig6a_gdp_world_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig6a_gdp_world_dark.png) [`svg`](visualisation/fig6a_gdp_world_dark.svg) [`json`](visualisation/fig6a_gdp_world_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6a_gdp_world_local_no_branding_dark.png) [`svg`](visualisation/fig6a_gdp_world_local_no_branding_dark.svg) [`json`](visualisation/fig6a_gdp_world_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6a_gdp_world_no_branding_dark.png) [`svg`](visualisation/fig6a_gdp_world_no_branding_dark.svg) [`json`](visualisation/fig6a_gdp_world_no_branding_dark.json)   

!["fig6a_gdp_world"](visualisation/fig6a_gdp_world.svg "fig6a_gdp_world")

  

!["fig6a_gdp_world_dark"](visualisation/fig6a_gdp_world_dark.svg "fig6a_gdp_world")

## Figure 6b  

Data: [`csv`](data/fig6b_gdp_ch.csv)  
GitHub: [fig6b_gdp_ch](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6b_gdp_ch_local.png) [`svg`](visualisation/fig6b_gdp_ch_local.svg) [`json`](visualisation/fig6b_gdp_ch_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig6b_gdp_ch.png) [`svg`](visualisation/fig6b_gdp_ch.svg) [`json`](visualisation/fig6b_gdp_ch.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6b_gdp_ch_local_no_branding.png) [`svg`](visualisation/fig6b_gdp_ch_local_no_branding.svg) [`json`](visualisation/fig6b_gdp_ch_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6b_gdp_ch_no_branding.png) [`svg`](visualisation/fig6b_gdp_ch_no_branding.svg) [`json`](visualisation/fig6b_gdp_ch_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6b_gdp_ch_local_dark.png) [`svg`](visualisation/fig6b_gdp_ch_local_dark.svg) [`json`](visualisation/fig6b_gdp_ch_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig6b_gdp_ch_dark.png) [`svg`](visualisation/fig6b_gdp_ch_dark.svg) [`json`](visualisation/fig6b_gdp_ch_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6b_gdp_ch_local_no_branding_dark.png) [`svg`](visualisation/fig6b_gdp_ch_local_no_branding_dark.svg) [`json`](visualisation/fig6b_gdp_ch_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6b_gdp_ch_no_branding_dark.png) [`svg`](visualisation/fig6b_gdp_ch_no_branding_dark.svg) [`json`](visualisation/fig6b_gdp_ch_no_branding_dark.json)   

!["fig6b_gdp_ch"](visualisation/fig6b_gdp_ch.svg "fig6b_gdp_ch")

  

!["fig6b_gdp_ch_dark"](visualisation/fig6b_gdp_ch_dark.svg "fig6b_gdp_ch")

## Figure 6b  

Data: [`csv`](data/fig6b_gdp_mz.csv)  
GitHub: [fig6b_gdp_mz](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6b_gdp_mz_local.png) [`svg`](visualisation/fig6b_gdp_mz_local.svg) [`json`](visualisation/fig6b_gdp_mz_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig6b_gdp_mz.png) [`svg`](visualisation/fig6b_gdp_mz.svg) [`json`](visualisation/fig6b_gdp_mz.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6b_gdp_mz_local_no_branding.png) [`svg`](visualisation/fig6b_gdp_mz_local_no_branding.svg) [`json`](visualisation/fig6b_gdp_mz_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6b_gdp_mz_no_branding.png) [`svg`](visualisation/fig6b_gdp_mz_no_branding.svg) [`json`](visualisation/fig6b_gdp_mz_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6b_gdp_mz_local_dark.png) [`svg`](visualisation/fig6b_gdp_mz_local_dark.svg) [`json`](visualisation/fig6b_gdp_mz_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig6b_gdp_mz_dark.png) [`svg`](visualisation/fig6b_gdp_mz_dark.svg) [`json`](visualisation/fig6b_gdp_mz_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6b_gdp_mz_local_no_branding_dark.png) [`svg`](visualisation/fig6b_gdp_mz_local_no_branding_dark.svg) [`json`](visualisation/fig6b_gdp_mz_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6b_gdp_mz_no_branding_dark.png) [`svg`](visualisation/fig6b_gdp_mz_no_branding_dark.svg) [`json`](visualisation/fig6b_gdp_mz_no_branding_dark.json)   

!["fig6b_gdp_mz"](visualisation/fig6b_gdp_mz.svg "fig6b_gdp_mz")

  

!["fig6b_gdp_mz_dark"](visualisation/fig6b_gdp_mz_dark.svg "fig6b_gdp_mz")

## Figure 7  

Data: [`csv`](data/fig7_co2_world.csv)  
GitHub: [fig7_co2_world](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7_co2_world_local.png) [`svg`](visualisation/fig7_co2_world_local.svg) [`json`](visualisation/fig7_co2_world_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig7_co2_world.png) [`svg`](visualisation/fig7_co2_world.svg) [`json`](visualisation/fig7_co2_world.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7_co2_world_local_no_branding.png) [`svg`](visualisation/fig7_co2_world_local_no_branding.svg) [`json`](visualisation/fig7_co2_world_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig7_co2_world_no_branding.png) [`svg`](visualisation/fig7_co2_world_no_branding.svg) [`json`](visualisation/fig7_co2_world_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7_co2_world_local_dark.png) [`svg`](visualisation/fig7_co2_world_local_dark.svg) [`json`](visualisation/fig7_co2_world_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig7_co2_world_dark.png) [`svg`](visualisation/fig7_co2_world_dark.svg) [`json`](visualisation/fig7_co2_world_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7_co2_world_local_no_branding_dark.png) [`svg`](visualisation/fig7_co2_world_local_no_branding_dark.svg) [`json`](visualisation/fig7_co2_world_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig7_co2_world_no_branding_dark.png) [`svg`](visualisation/fig7_co2_world_no_branding_dark.svg) [`json`](visualisation/fig7_co2_world_no_branding_dark.json)   

!["fig7_co2_world"](visualisation/fig7_co2_world.svg "fig7_co2_world")

  

!["fig7_co2_world_dark"](visualisation/fig7_co2_world_dark.svg "fig7_co2_world")

## Figure 7b  

Data: [`csv`](data/fig7b_co2_ch.csv)  
GitHub: [fig7b_co2_ch](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7b_co2_ch_local.png) [`svg`](visualisation/fig7b_co2_ch_local.svg) [`json`](visualisation/fig7b_co2_ch_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig7b_co2_ch.png) [`svg`](visualisation/fig7b_co2_ch.svg) [`json`](visualisation/fig7b_co2_ch.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7b_co2_ch_local_no_branding.png) [`svg`](visualisation/fig7b_co2_ch_local_no_branding.svg) [`json`](visualisation/fig7b_co2_ch_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig7b_co2_ch_no_branding.png) [`svg`](visualisation/fig7b_co2_ch_no_branding.svg) [`json`](visualisation/fig7b_co2_ch_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7b_co2_ch_local_dark.png) [`svg`](visualisation/fig7b_co2_ch_local_dark.svg) [`json`](visualisation/fig7b_co2_ch_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig7b_co2_ch_dark.png) [`svg`](visualisation/fig7b_co2_ch_dark.svg) [`json`](visualisation/fig7b_co2_ch_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7b_co2_ch_local_no_branding_dark.png) [`svg`](visualisation/fig7b_co2_ch_local_no_branding_dark.svg) [`json`](visualisation/fig7b_co2_ch_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig7b_co2_ch_no_branding_dark.png) [`svg`](visualisation/fig7b_co2_ch_no_branding_dark.svg) [`json`](visualisation/fig7b_co2_ch_no_branding_dark.json)   

!["fig7b_co2_ch"](visualisation/fig7b_co2_ch.svg "fig7b_co2_ch")

  

!["fig7b_co2_ch_dark"](visualisation/fig7b_co2_ch_dark.svg "fig7b_co2_ch")

## Figure 7c  

Data: [`csv`](data/fig7c_co2_mz.csv)  
GitHub: [fig7c_co2_mz](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7c_co2_mz_local.png) [`svg`](visualisation/fig7c_co2_mz_local.svg) [`json`](visualisation/fig7c_co2_mz_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig7c_co2_mz.png) [`svg`](visualisation/fig7c_co2_mz.svg) [`json`](visualisation/fig7c_co2_mz.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7c_co2_mz_local_no_branding.png) [`svg`](visualisation/fig7c_co2_mz_local_no_branding.svg) [`json`](visualisation/fig7c_co2_mz_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig7c_co2_mz_no_branding.png) [`svg`](visualisation/fig7c_co2_mz_no_branding.svg) [`json`](visualisation/fig7c_co2_mz_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7c_co2_mz_local_dark.png) [`svg`](visualisation/fig7c_co2_mz_local_dark.svg) [`json`](visualisation/fig7c_co2_mz_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig7c_co2_mz_dark.png) [`svg`](visualisation/fig7c_co2_mz_dark.svg) [`json`](visualisation/fig7c_co2_mz_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7c_co2_mz_local_no_branding_dark.png) [`svg`](visualisation/fig7c_co2_mz_local_no_branding_dark.svg) [`json`](visualisation/fig7c_co2_mz_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig7c_co2_mz_no_branding_dark.png) [`svg`](visualisation/fig7c_co2_mz_no_branding_dark.svg) [`json`](visualisation/fig7c_co2_mz_no_branding_dark.json)   

!["fig7c_co2_mz"](visualisation/fig7c_co2_mz.svg "fig7c_co2_mz")

  

!["fig7c_co2_mz_dark"](visualisation/fig7c_co2_mz_dark.svg "fig7c_co2_mz")

## Figure 8a  

Data: [`csv`](data/fig8a_gpr_world.csv)  
GitHub: [fig8a_gpr_world](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8a_gpr_world_local.png) [`svg`](visualisation/fig8a_gpr_world_local.svg) [`json`](visualisation/fig8a_gpr_world_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig8a_gpr_world.png) [`svg`](visualisation/fig8a_gpr_world.svg) [`json`](visualisation/fig8a_gpr_world.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8a_gpr_world_local_no_branding.png) [`svg`](visualisation/fig8a_gpr_world_local_no_branding.svg) [`json`](visualisation/fig8a_gpr_world_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig8a_gpr_world_no_branding.png) [`svg`](visualisation/fig8a_gpr_world_no_branding.svg) [`json`](visualisation/fig8a_gpr_world_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8a_gpr_world_local_dark.png) [`svg`](visualisation/fig8a_gpr_world_local_dark.svg) [`json`](visualisation/fig8a_gpr_world_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig8a_gpr_world_dark.png) [`svg`](visualisation/fig8a_gpr_world_dark.svg) [`json`](visualisation/fig8a_gpr_world_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8a_gpr_world_local_no_branding_dark.png) [`svg`](visualisation/fig8a_gpr_world_local_no_branding_dark.svg) [`json`](visualisation/fig8a_gpr_world_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig8a_gpr_world_no_branding_dark.png) [`svg`](visualisation/fig8a_gpr_world_no_branding_dark.svg) [`json`](visualisation/fig8a_gpr_world_no_branding_dark.json)   

!["fig8a_gpr_world"](visualisation/fig8a_gpr_world.svg "fig8a_gpr_world")

  

!["fig8a_gpr_world_dark"](visualisation/fig8a_gpr_world_dark.svg "fig8a_gpr_world")

## Figure 8b  

Data: [`csv`](data/fig8b_gpr_ch.csv)  
GitHub: [fig8b_gpr_ch](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8b_gpr_ch_local.png) [`svg`](visualisation/fig8b_gpr_ch_local.svg) [`json`](visualisation/fig8b_gpr_ch_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig8b_gpr_ch.png) [`svg`](visualisation/fig8b_gpr_ch.svg) [`json`](visualisation/fig8b_gpr_ch.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8b_gpr_ch_local_no_branding.png) [`svg`](visualisation/fig8b_gpr_ch_local_no_branding.svg) [`json`](visualisation/fig8b_gpr_ch_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig8b_gpr_ch_no_branding.png) [`svg`](visualisation/fig8b_gpr_ch_no_branding.svg) [`json`](visualisation/fig8b_gpr_ch_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8b_gpr_ch_local_dark.png) [`svg`](visualisation/fig8b_gpr_ch_local_dark.svg) [`json`](visualisation/fig8b_gpr_ch_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig8b_gpr_ch_dark.png) [`svg`](visualisation/fig8b_gpr_ch_dark.svg) [`json`](visualisation/fig8b_gpr_ch_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8b_gpr_ch_local_no_branding_dark.png) [`svg`](visualisation/fig8b_gpr_ch_local_no_branding_dark.svg) [`json`](visualisation/fig8b_gpr_ch_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig8b_gpr_ch_no_branding_dark.png) [`svg`](visualisation/fig8b_gpr_ch_no_branding_dark.svg) [`json`](visualisation/fig8b_gpr_ch_no_branding_dark.json)   

!["fig8b_gpr_ch"](visualisation/fig8b_gpr_ch.svg "fig8b_gpr_ch")

  

!["fig8b_gpr_ch_dark"](visualisation/fig8b_gpr_ch_dark.svg "fig8b_gpr_ch")

## Figure 8c  

Data: [`csv`](data/fig8c_gpr_mz.csv)  
GitHub: [fig8c_gpr_mz](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8c_gpr_mz_local.png) [`svg`](visualisation/fig8c_gpr_mz_local.svg) [`json`](visualisation/fig8c_gpr_mz_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig8c_gpr_mz.png) [`svg`](visualisation/fig8c_gpr_mz.svg) [`json`](visualisation/fig8c_gpr_mz.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8c_gpr_mz_local_no_branding.png) [`svg`](visualisation/fig8c_gpr_mz_local_no_branding.svg) [`json`](visualisation/fig8c_gpr_mz_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig8c_gpr_mz_no_branding.png) [`svg`](visualisation/fig8c_gpr_mz_no_branding.svg) [`json`](visualisation/fig8c_gpr_mz_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8c_gpr_mz_local_dark.png) [`svg`](visualisation/fig8c_gpr_mz_local_dark.svg) [`json`](visualisation/fig8c_gpr_mz_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig8c_gpr_mz_dark.png) [`svg`](visualisation/fig8c_gpr_mz_dark.svg) [`json`](visualisation/fig8c_gpr_mz_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8c_gpr_mz_local_no_branding_dark.png) [`svg`](visualisation/fig8c_gpr_mz_local_no_branding_dark.svg) [`json`](visualisation/fig8c_gpr_mz_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig8c_gpr_mz_no_branding_dark.png) [`svg`](visualisation/fig8c_gpr_mz_no_branding_dark.svg) [`json`](visualisation/fig8c_gpr_mz_no_branding_dark.json)   

!["fig8c_gpr_mz"](visualisation/fig8c_gpr_mz.svg "fig8c_gpr_mz")

  

!["fig8c_gpr_mz_dark"](visualisation/fig8c_gpr_mz_dark.svg "fig8c_gpr_mz")

## Figure 9a  

Data: [`csv`](data/fig9a_gdp_map.csv)  
GitHub: [fig9a_gdp_map](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9a_gdp_map_local.png) [`svg`](visualisation/fig9a_gdp_map_local.svg) [`json`](visualisation/fig9a_gdp_map_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig9a_gdp_map.png) [`svg`](visualisation/fig9a_gdp_map.svg) [`json`](visualisation/fig9a_gdp_map.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9a_gdp_map_local_no_branding.png) [`svg`](visualisation/fig9a_gdp_map_local_no_branding.svg) [`json`](visualisation/fig9a_gdp_map_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig9a_gdp_map_no_branding.png) [`svg`](visualisation/fig9a_gdp_map_no_branding.svg) [`json`](visualisation/fig9a_gdp_map_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9a_gdp_map_local_dark.png) [`svg`](visualisation/fig9a_gdp_map_local_dark.svg) [`json`](visualisation/fig9a_gdp_map_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig9a_gdp_map_dark.png) [`svg`](visualisation/fig9a_gdp_map_dark.svg) [`json`](visualisation/fig9a_gdp_map_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9a_gdp_map_local_no_branding_dark.png) [`svg`](visualisation/fig9a_gdp_map_local_no_branding_dark.svg) [`json`](visualisation/fig9a_gdp_map_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig9a_gdp_map_no_branding_dark.png) [`svg`](visualisation/fig9a_gdp_map_no_branding_dark.svg) [`json`](visualisation/fig9a_gdp_map_no_branding_dark.json)   

!["fig9a_gdp_map"](visualisation/fig9a_gdp_map.svg "fig9a_gdp_map")

  

!["fig9a_gdp_map_dark"](visualisation/fig9a_gdp_map_dark.svg "fig9a_gdp_map")

## Figure 9b  

Data: [`csv`](data/fig9b_co2_map.csv)  
GitHub: [fig9b_co2_map](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9b_co2_map_local.png) [`svg`](visualisation/fig9b_co2_map_local.svg) [`json`](visualisation/fig9b_co2_map_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig9b_co2_map.png) [`svg`](visualisation/fig9b_co2_map.svg) [`json`](visualisation/fig9b_co2_map.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9b_co2_map_local_no_branding.png) [`svg`](visualisation/fig9b_co2_map_local_no_branding.svg) [`json`](visualisation/fig9b_co2_map_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig9b_co2_map_no_branding.png) [`svg`](visualisation/fig9b_co2_map_no_branding.svg) [`json`](visualisation/fig9b_co2_map_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9b_co2_map_local_dark.png) [`svg`](visualisation/fig9b_co2_map_local_dark.svg) [`json`](visualisation/fig9b_co2_map_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig9b_co2_map_dark.png) [`svg`](visualisation/fig9b_co2_map_dark.svg) [`json`](visualisation/fig9b_co2_map_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9b_co2_map_local_no_branding_dark.png) [`svg`](visualisation/fig9b_co2_map_local_no_branding_dark.svg) [`json`](visualisation/fig9b_co2_map_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig9b_co2_map_no_branding_dark.png) [`svg`](visualisation/fig9b_co2_map_no_branding_dark.svg) [`json`](visualisation/fig9b_co2_map_no_branding_dark.json)   

!["fig9b_co2_map"](visualisation/fig9b_co2_map.svg "fig9b_co2_map")

  

!["fig9b_co2_map_dark"](visualisation/fig9b_co2_map_dark.svg "fig9b_co2_map")

## Figure 9c  

Data: [`csv`](data/fig9c_gpr_map.csv)  
GitHub: [fig9c_gpr_map](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9c_gpr_map_local.png) [`svg`](visualisation/fig9c_gpr_map_local.svg) [`json`](visualisation/fig9c_gpr_map_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig9c_gpr_map.png) [`svg`](visualisation/fig9c_gpr_map.svg) [`json`](visualisation/fig9c_gpr_map.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9c_gpr_map_local_no_branding.png) [`svg`](visualisation/fig9c_gpr_map_local_no_branding.svg) [`json`](visualisation/fig9c_gpr_map_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig9c_gpr_map_no_branding.png) [`svg`](visualisation/fig9c_gpr_map_no_branding.svg) [`json`](visualisation/fig9c_gpr_map_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9c_gpr_map_local_dark.png) [`svg`](visualisation/fig9c_gpr_map_local_dark.svg) [`json`](visualisation/fig9c_gpr_map_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig9c_gpr_map_dark.png) [`svg`](visualisation/fig9c_gpr_map_dark.svg) [`json`](visualisation/fig9c_gpr_map_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9c_gpr_map_local_no_branding_dark.png) [`svg`](visualisation/fig9c_gpr_map_local_no_branding_dark.svg) [`json`](visualisation/fig9c_gpr_map_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig9c_gpr_map_no_branding_dark.png) [`svg`](visualisation/fig9c_gpr_map_no_branding_dark.svg) [`json`](visualisation/fig9c_gpr_map_no_branding_dark.json)   

!["fig9c_gpr_map"](visualisation/fig9c_gpr_map.svg "fig9c_gpr_map")

  

!["fig9c_gpr_map_dark"](visualisation/fig9c_gpr_map_dark.svg "fig9c_gpr_map")

